# 上证指数涨跌预测

Install required moduels      

    pip install numpy
    pip install scipy
    pip install scikit-learn

In [4]:
!pip install scipy

In [5]:
!pip install scikit-learn

In [6]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn import cross_validation

c:\users\hcche\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [19]:
import peforth
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [8]:
data=pd.read_csv('000777.csv',encoding='gbk',parse_dates=[0],index_col=0)

In [9]:
# pdb.set_trace() # data 本來是新的在上面
data.sort_index(0,ascending=True,inplace=True) # 排序 sort 過後變成最舊的在上面

In [12]:
data.index

DatetimeIndex(['1997-06-16', '1997-06-17', '1997-06-18', '1997-06-19',
               '1997-06-20', '1997-06-23', '1997-06-24', '1997-06-25',
               '1997-06-26', '1997-06-27',
               ...
               '2017-01-09', '2017-01-10', '2017-01-11', '2017-01-12',
               '2017-01-13', '2017-01-16', '2017-01-17', '2017-01-18',
               '2017-01-19', '2017-01-20'],
              dtype='datetime64[ns]', name='日期', length=4752, freq=None)

In [15]:
data.query('index < "1997-06-20"')

,股票代码,名称,收盘价,最高价,最低价,开盘价,前收盘,涨跌额,涨跌幅,换手率,成交量,成交金额,总市值,流通市值
日期,,,,,,,,,,,,,,
1997-06-16,'000777,中核苏阀,0.0,0.0,0.0,0.0,5.33,None,None,None,0,0.0,239850000.0,0.0
1997-06-17,'000777,中核苏阀,0.0,0.0,0.0,0.0,5.33,None,None,None,0,0.0,239850000.0,0.0
1997-06-18,'000777,中核苏阀,0.0,0.0,0.0,0.0,5.33,None,None,None,0,0.0,239850000.0,0.0
1997-06-19,'000777,中核苏阀,0.0,0.0,0.0,0.0,5.33,None,None,None,0,0.0,239850000.0,0.0


In [16]:
# raw data has 20 years long, but here only analysis the last 150 days of a chosen day
dayfeature=150
featurenum=5*dayfeature  # 取五個有關係的欄位，特徵值。
x=np.zeros((data.shape[0]-dayfeature,featurenum+1))
y=np.zeros((data.shape[0]-dayfeature));

In [26]:
peforth.push(data).dictate('constant data');
peforth.push(dayfeature).dictate('constant dayfeature');
peforth.push(x).dictate('value x');
peforth.push(y).dictate('value y');

In [27]:
%f data :> shape . cr
%f data :> shape[0] . cr
%f data :> shape[0]-v('dayfeature') . cr
peforth.ok();

(4752, 14)
4752
4602
x . cr
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
OK y . cr 
[0. 0. 0. ... 0. 0. 0.]
OK x :> shape 
OK . cr
(4602, 751)
OK y :> shape . cr
(4602,)
OK 150 5 * 1 + . cr
751
OK exit
OK 

In [29]:
for i in range(0,data.shape[0]-dayfeature):
    x[i,0:featurenum]=np.array(data[i:i+dayfeature] \
          [[u'收盘价',u'最高价',u'最低价',u'开盘价',u'成交量']]).reshape((1,featurenum))
    # x[i,featurenum]=data.ix[i+dayfeature][u'开盘价']  # ix deprecated
    x[i,featurenum]=data.iloc[i+dayfeature][u'开盘价']

In [30]:
for i in range(0,data.shape[0]-dayfeature):
    if data.ix[i+dayfeature][u'收盘价']>=data.ix[i+dayfeature][u'开盘价']:
        y[i]=1
    else:
        y[i]=0          

In [31]:
clf=svm.SVC(kernel='poly')  # so what is SVM? See "11-提交-监督学习 - 课程导学.pdf"
result = []

In [ ]:
for i in range(100):
    x_train, x_test, y_train, y_test = \
                cross_validation.train_test_split(x, y, test_size = 0.2)
    clf.fit(x_train, y_train)
    result.append(np.mean(y_test == clf.predict(x_test)))

In [33]:
print("svm classifier accuacy:")
print(result)

svm classifier accuacy:
[0.47122692725298587, 0.5233441910966341, 0.5504885993485342, 0.50814332247557, 0.5298588490770901, 0.4549402823018458, 0.5331161780673181, 0.5135722041259501, 0.4744842562432139, 0.5515743756786102]
